Import Libraries

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import *

Create DataFrame

In [0]:
df = spark.read.load('/FileStore/tables/googleplaystore.csv',
                     format='csv',
                     header='true',
                     inferschema='true',
                     )
df.show(3)

+--------------------+--------------+------+-------+----+----------+----+-----+--------------+--------------------+----------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|  Installs|Type|Price|Content Rating|              Genres|    Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+----------+----+-----+--------------+--------------------+----------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|   10,000+|Free|    0|      Everyone|        Art & Design| January 7, 2018|      1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|  500,000+|Free|    0|      Everyone|Art & Design;Pret...|January 15, 2018|      2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M|5,000,000+|Free|    0|      Everyone|        Art & Design|  August 1, 2018|      1.2.4|4.0.3 and up|
+--------------------+--------------+------+-------+

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Data Cleaning

In [0]:
df = df.withColumn('Reviews', col('Reviews').cast(IntegerType()))\
      .withColumn('Installs', regexp_replace(col('Installs'), '[^0-9]', ''))\
      .withColumn('Installs', col('Installs').cast(IntegerType()))\
      .withColumn('Price', regexp_replace(col('Price'), '[$]', ''))\
      .withColumn('Price', col('Price').cast(IntegerType()))\
      .withColumn('Last Updated', date_format(to_date("Last Updated", "MMMM d, yyyy"), "dd/MM/yyyy"))\
      .withColumn('Rating', col('Rating').cast(FloatType()))\
      .withColumn('Type', regexp_replace(col('Type'), '[^Free|Paid]', ''))\
      .withColumn('Genres', regexp_replace(col('Genres'), ';(.*)', ''))\
      .withColumnRenamed('Content Rating', 'Content_Rating')\
      .withColumnRenamed('Last Updated', 'Last_Updated')\

df.show(2)
df.printSchema()


+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content_Rating|      Genres|Last_Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|   10000|Free|    0|      Everyone|Art & Design|  07/01/2018|      1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|  500000|Free|    0|      Everyone|Art & Design|  15/01/2018|      2.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+------------+-----------+------------+
only showing top 2 rows

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = 

In [0]:
df = df.drop('Current Ver', 'Android Ver')
df = df.dropna()
df.show(10)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content_Rating|      Genres|Last_Updated|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|   10000|Free|    0|      Everyone|Art & Design|  07/01/2018|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|  500000|Free|    0|      Everyone|Art & Design|  15/01/2018|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5000000|Free|    0|      Everyone|Art & Design|  01/08/2018|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644| 25M|50000000|Free|    0|          Teen|Art & Design|  08/06/2018|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|2.8M|  100000|Free|    0|      Everyone|Art & Design|  20/06/2018|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    

Convert to View for Analysis of Data using SQL

In [0]:
df.createOrReplaceTempView('Apps')

In [0]:
%sql
select * from apps limit 10

App,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Genres,Last_Updated
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,10000,Free,0,Everyone,Art & Design,07/01/2018
Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0,Everyone,Art & Design,15/01/2018
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,8.7M,5000000,Free,0,Everyone,Art & Design,01/08/2018
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,50000000,Free,0,Teen,Art & Design,08/06/2018
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,100000,Free,0,Everyone,Art & Design,20/06/2018
Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,50000,Free,0,Everyone,Art & Design,26/03/2017
Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,50000,Free,0,Everyone,Art & Design,26/04/2018
Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,1000000,Free,0,Everyone,Art & Design,14/06/2018
Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,1000000,Free,0,Everyone,Art & Design,20/09/2017
Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,10000,Free,0,Everyone,Art & Design,03/07/2018


Top 10 Apps based on number of Reviews

In [0]:
%sql
select distinct App, Max(Reviews) as Total_Reviews from Apps group by App order by Total_Reviews desc limit 10

App,Total_Reviews
Facebook,78158306
WhatsApp Messenger,69119316
Instagram,66577446
Messenger – Text and Video Chat for Free,56646578
Clash of Clans,44893888
Clean Master- Space Cleaner & Antivirus,42916526
Subway Surfers,27725352
YouTube,25655305
"Security Master - Antivirus, VPN, AppLock, Booster",24900999
Clash Royale,23136735


Databricks visualization. Run in Databricks to view.

Top 10 Apps based on number of Installs

In [0]:
%sql
select App, Installs from Apps group by App, Installs order by Installs desc limit 10

App,Installs
Google Street View,1000000000
Facebook,1000000000
Google Play Games,1000000000
Google Drive,1000000000
Messenger – Text and Video Chat for Free,1000000000
Skype - free IM & video calls,1000000000
Google News,1000000000
Gmail,1000000000
Google,1000000000
Google Chrome: Fast & Secure,1000000000


Number of apps in each Content_Rating

In [0]:
%sql
select distinct Content_Rating, count(Content_Rating) as count from apps group by 1 having count>2 order by count


Content_Rating,count
Adults only 18+,3
Everyone 10+,397
Mature 17+,460
Teen,1084
Everyone,7419


Databricks visualization. Run in Databricks to view.

Group by "Genres" and calculate the minimum, maximum, and average ratings for apps within each genre.

In [0]:
%sql
SELECT Genres,
       round(MIN(Rating),2) AS MinRating,
       round(MAX(Rating),2) AS MaxRating,
       round(AVG(Rating),2) AS AvgRating,
       count(App) as TotalApps
FROM apps
GROUP BY Genres
ORDER BY AvgRating DESC;

Genres,MinRating,MaxRating,AvgRating,TotalApps
Events,2.9,5.0,4.44,45
Word,3.8,4.9,4.41,28
Puzzle,2.9,5.0,4.39,148
Art & Design,3.2,5.0,4.35,65
Personalization,2.5,5.0,4.34,314
Books & Reference,2.7,5.0,4.34,180
Education,1.0,5.0,4.31,563
Arcade,3.1,5.0,4.31,223
Music & Audio,4.3,4.3,4.3,1
Board,2.8,4.8,4.3,60


Databricks visualization. Run in Databricks to view.

Average rating for free vs paid apps:

In [0]:
%sql
select Type, count(App) as Total_Apps, round(AVG(Rating), 2) as AvgRating from Apps group by Type

Type,Total_Apps,AvgRating
Free,8717,4.19
Paid,647,4.27


Databricks visualization. Run in Databricks to view.

Top App in each Category based on number of Installs

In [0]:
%sql
SELECT Category, first_value(App) AS TopApp, max(Installs)
FROM apps
GROUP BY Category

Category,TopApp,max(Installs)
ART_AND_DESIGN,Photo Editor & Candy Camera & Grid & ScrapBook,50000000
AUTO_AND_VEHICLES,Monster Truck Stunt 3D 2019,10000000
BEAUTY,Hush - Beauty for Everyone,10000000
BOOKS_AND_REFERENCE,Wattpad 📖 Free Books,1000000000
BUSINESS,Visual Voicemail by MetroPCS,100000000
COMICS,LINE WEBTOON - Free Comics,10000000
COMMUNICATION,Messenger – Text and Video Chat for Free,1000000000
DATING,"Moco+ - Chat, Meet People",10000000
EDUCATION,Duolingo: Learn Languages Free,100000000
ENTERTAINMENT,Netflix,1000000000


Databricks visualization. Run in Databricks to view.

Exporting App data based on Type in csv files

In [0]:
free_apps_df = df.filter(df.Type == 'Free')
paid_apps_df = df.filter(df.Type == 'Paid')

free_apps_df.write.format('csv').option('header', 'true').save("/FileStore/free_apps.csv")
paid_apps_df.write.format('csv').option('header', 'true').save("/FileStore/paid_apps.csv")